In [1]:
import csv
import json
from urllib.request import urlopen

output_filename = "tocantins.csv"

url_casos_totais = "http://integra.saude.to.gov.br/Api/GetReleasesCovid?id=247"
url_obitos_totais = "http://integra.saude.to.gov.br/Api/GetReleasesCovid?id=249"
url_municipios = "http://integra.saude.to.gov.br/Api/GetReleasesCovid?id=250"

def download_data(url):
    response = json.loads(urlopen(url).read())
    data = json.loads(response["Retorno"])
    header = [item["colName"] for item in data["metadata"]]
    values = data["resultset"]
    result = []
    for row in values:
        row = dict(zip(header, row))
        result.append(row)
    return result

casos_totais = download_data(url_casos_totais)
obitos_totais = download_data(url_obitos_totais)
municipios = download_data(url_municipios)

with open(output_filename, mode="w", encoding="utf-8", newline="") as fobj:
    writer = csv.DictWriter(fobj, fieldnames=["municipio", "confirmados", "obitos"])
    writer.writeheader()
    writer.writerow(
        {
            "municipio": "TOTAL NO ESTADO",
            "confirmados": casos_totais[0]["Casos Confirmados"],
            "obitos": obitos_totais[0]["Óbitos"],
        }
    )
    for municipio in municipios:
        nome = municipio["Município"]
        if nome == "Fortaleza do Tabocão":
            nome = "Tabocão"
        writer.writerow(
            {
                "municipio": nome,
                "confirmados": municipio["Casos Confirmados"],
                "obitos": municipio["Óbitos"],
            }
        )